<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Analysis-of-Extent-of-Cloned-Codes" data-toc-modified-id="Analysis-of-Extent-of-Cloned-Codes-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Analysis of Extent of Cloned Codes</a></span></li><li><span><a href="#Percentage-of-Inpatient-/-outpatient-claims-are-cloned-and-sum-reimbursement" data-toc-modified-id="Percentage-of-Inpatient-/-outpatient-claims-are-cloned-and-sum-reimbursement-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Percentage of Inpatient / outpatient claims are cloned and sum reimbursement</a></span></li><li><span><a href="#which-chronic-conditions-are-most-frequently-cloned?-use-apriori-frequent-patterns" data-toc-modified-id="which-chronic-conditions-are-most-frequently-cloned?-use-apriori-frequent-patterns-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>which chronic conditions are most frequently cloned? use apriori frequent patterns</a></span></li><li><span><a href="#Distribution-of-how-many-times-cloning-occurs-(number-of-cloned-rows)" data-toc-modified-id="Distribution-of-how-many-times-cloning-occurs-(number-of-cloned-rows)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Distribution of how many times cloning occurs (number of cloned rows)</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 10)
sns.set(style = "whitegrid")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# matplotlib params
size=15
params = {'legend.fontsize': 'large',
          'figure.figsize': (20,8),
          'axes.labelsize': size*1.1,
          'axes.titlesize': size*2,
          'xtick.labelsize': size*1.1,
          'ytick.labelsize': size*1.1,
          'axes.titlepad': 25,
          'figure.titlesize': size*2,
          'legend.fontsize': size,
            'legend.title_fontsize': size*1.1
         }

plt.rcParams.update(params)

sns.set_style("whitegrid")

In [3]:
# read in dataframes from binaries
trB= pd.read_pickle("./data/trB.pkl") # Train Beneficiaries 
trO= pd.read_pickle("./data/trO.pkl") # Train Outpatient Claims
trI= pd.read_pickle("./data/trI.pkl") # Train Inpatient Claims
trF= pd.read_pickle("./data/trF.pkl") # Train Provider Fraud

# Analysis of Extent of Cloned Codes

In [4]:
# Inpatient Cloned Diagnosis Codes

diag=['ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10']

trI[trI.duplicated(subset=diag , keep=False)].dropna(subset=diag, how='all')
# 1550 inpatient claims have duplicate Diagnosis codes

,index,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
34,34,BENE11093,CLM32146,2009-01-03,2009-01-09,PRV51842,20000,PHY341192,PHY355416,None,2009-01-03,4439,1068.0,2009-01-09,255,99662,2639,E8782,920,V854,4280,E9320,04185,412,42732,3950.0,2762.0,NaN,NaN,NaN,None
35,35,BENE11093,CLM82202,2009-01-03,2009-01-12,PRV51842,20000,PHY341192,PHY355416,None,2009-01-03,4439,1068.0,2009-01-09,255,99662,2639,E8782,920,V854,4280,E9320,04185,412,42732,3950.0,2762.0,NaN,NaN,NaN,None
38,38,BENE11101,CLM46868,2009-04-14,2009-04-18,PRV51164,7000,PHY318943,PHY358329,None,2009-04-14,486,1068.0,2009-04-18,190,5070,5849,V0991,7863,71946,49121,2859,496,4589,None,3324.0,NaN,NaN,NaN,NaN,None
52,52,BENE11178,CLM40640,2009-03-01,2009-03-08,PRV57437,2000,PHY323517,None,None,2009-03-01,311,1068.0,2009-03-08,884,3071,53081,2859,261,30500,V850,5990,29689,None,None,NaN,NaN,NaN,NaN,NaN,None
54,54,BENE11182,CLM73312,2009-10-25,2009-10-29,PRV55903,4000,PHY400539,PHY376585,None,2009-10-25,431,1068.0,2009-10-29,022,43491,4019,29590,7852,412,7843,311,53081,71690,None,8872.0,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40361,40361,BENE158810,CLM39286,2009-02-20,2009-02-21,PRV53762,2000,PHY378430,None,None,2009-02-20,29181,1068.0,2009-02-21,894,29181,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None
40388,40388,BENE158883,CLM35368,2009-01-24,2009-01-27,PRV55435,10000,PHY403345,PHY403345,None,2009-01-24,79902,1068.0,2009-01-27,868,0389,99592,48241,78552,5859,2809,5856,2768,27651,None,9905.0,NaN,NaN,NaN,NaN,None
40411,40411,BENE158947,CLM43434,2009-03-20,2009-03-25,PRV57204,0,PHY367487,PHY329719,None,2009-03-20,486,1068.0,2009-03-25,182,4809,5990,25000,42823,42822,2724,34400,7392,4280,None,9952.0,NaN,NaN,NaN,NaN,None
40413,40413,BENE158952,CLM36936,2009-02-04,2009-02-07,PRV54342,7000,PHY417489,PHY353365,None,2009-02-04,78650,1068.0,2009-02-07,443,57410,36250,53081,5739,42789,4280,4293,4019,5531,None,5123.0,NaN,NaN,NaN,NaN,None


In [5]:
# Inpatient Cloned Procedure Codes
proc=['ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6']

len(trI[trI.duplicated(subset=proc, keep=False)].dropna(subset=proc, how='all'))
# 20367 inpatient claims have duplicate Procedure codes

20367

In [13]:
# Inpatient Cloned Diagnosis and Procedure Codes
trIclon = trI[trI.duplicated(subset=proc+diag, keep=False)].dropna(subset=proc+diag, how='all')
# 1507 inpatient claims have duplicate entierty of Diagnostic and Procedure codes

In [84]:
trI[trI.duplicated(subset=proc+diag, keep=False)].astype(str).replace(['nan','NaN', None, 'None'], 0)

,index,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
34,34,BENE11093,CLM32146,2009-01-03,2009-01-09,PRV51842,20000,PHY341192,PHY355416,0,2009-01-03,4439,1068.0,2009-01-09,255,99662,2639,E8782,920,V854,4280,E9320,04185,412,42732,3950.0,2762.0,0,0,0,0
35,35,BENE11093,CLM82202,2009-01-03,2009-01-12,PRV51842,20000,PHY341192,PHY355416,0,2009-01-03,4439,1068.0,2009-01-09,255,99662,2639,E8782,920,V854,4280,E9320,04185,412,42732,3950.0,2762.0,0,0,0,0
38,38,BENE11101,CLM46868,2009-04-14,2009-04-18,PRV51164,7000,PHY318943,PHY358329,0,2009-04-14,486,1068.0,2009-04-18,190,5070,5849,V0991,7863,71946,49121,2859,496,4589,0,3324.0,0,0,0,0,0
52,52,BENE11178,CLM40640,2009-03-01,2009-03-08,PRV57437,2000,PHY323517,0,0,2009-03-01,311,1068.0,2009-03-08,884,3071,53081,2859,261,30500,V850,5990,29689,0,0,0,0,0,0,0,0
54,54,BENE11182,CLM73312,2009-10-25,2009-10-29,PRV55903,4000,PHY400539,PHY376585,0,2009-10-25,431,1068.0,2009-10-29,022,43491,4019,29590,7852,412,7843,311,53081,71690,0,8872.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40361,40361,BENE158810,CLM39286,2009-02-20,2009-02-21,PRV53762,2000,PHY378430,0,0,2009-02-20,29181,1068.0,2009-02-21,894,29181,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40388,40388,BENE158883,CLM35368,2009-01-24,2009-01-27,PRV55435,10000,PHY403345,PHY403345,0,2009-01-24,79902,1068.0,2009-01-27,868,0389,99592,48241,78552,5859,2809,5856,2768,27651,0,9905.0,0,0,0,0,0
40411,40411,BENE158947,CLM43434,2009-03-20,2009-03-25,PRV57204,0,PHY367487,PHY329719,0,2009-03-20,486,1068.0,2009-03-25,182,4809,5990,25000,42823,42822,2724,34400,7392,4280,0,9952.0,0,0,0,0,0
40413,40413,BENE158952,CLM36936,2009-02-04,2009-02-07,PRV54342,7000,PHY417489,PHY353365,0,2009-02-04,78650,1068.0,2009-02-07,443,57410,36250,53081,5739,42789,4280,4293,4019,5531,0,5123.0,0,0,0,0,0


In [6]:
# Outpatient Cloned Diagnosis Codes
diag=['ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10']

trO[trO.duplicated(subset=diag , keep=False)].dropna(subset=diag, how='all')
# 242613 outpatient claims have duplicate Diagnosis codes

,index,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode
1,1,BENE11003,CLM189947,2009-02-12,2009-02-12,PRV57610,80,PHY362868,None,None,6115,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,None,0,79380
2,2,BENE11003,CLM438021,2009-06-27,2009-06-27,PRV57595,10,PHY328821,None,None,2723,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,None,0,None
3,3,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,None,None,71988,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,None,0,None
5,5,BENE11004,CLM173224,2009-02-03,2009-02-03,PRV56011,20,PHY339887,None,None,20381,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,None,0,None
9,9,BENE11004,CLM339500,2009-05-04,2009-05-16,PRV56011,500,PHY412904,None,PHY396473,7237,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,None,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517724,517724,BENE159198,CLM111191,2008-12-23,2009-01-12,PRV53676,60,PHY361063,PHY361063,None,29655,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,None,0,3009
517726,517726,BENE159198,CLM189011,2009-02-11,2009-02-11,PRV53672,3300,PHY390366,None,None,99673,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,None,0,None
517727,517727,BENE159198,CLM255268,2009-03-19,2009-03-19,PRV53672,70,PHY317739,PHY317739,PHY423886,5929,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,None,0,None
517729,517729,BENE159198,CLM310720,2009-04-18,2009-05-08,PRV53670,0,PHY329971,None,None,29561,V5869,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,None,0,29570


In [7]:
# Outpatient Cloned Procedure Codes
proc=['ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6']

len(trO[trO.duplicated(subset=proc, keep=False)].dropna(subset=proc, how='all'))
# 82 outpatient claims have duplicate Procedure codes

82

In [14]:
# Outpatient Cloned Diagnosis and Procedure Codes

trOclon = trO[trO.duplicated(subset=proc+diag, keep=False)].dropna(subset=proc+diag, how='all')
# 242603 Outpatient claims have duplicate entierty of Diagnostic and Procedure codes

# Percentage of Inpatient / outpatient claims are cloned and sum reimbursement

In [15]:
len(trIclon) / len(trI)
# 1507 claims, 3.7% inpatient clones are cloned
trIclon['InscClaimAmtReimbursed'].sum()
# reimbursements sum to 15,064,000

15064000

In [16]:
len(trOclon) / len(trO)
# 47% outpatient clones are cloned (however, there is little likelyhood that these are fraudulent as few codes for each claim)
trOclon['InscClaimAmtReimbursed'].sum()
# reimbursements sum to 60,111,480

60111480

# which chronic conditions are most frequently cloned? use apriori frequent patterns
unsuprisingly, they are the most common, IschemicHeart, Diabetes, HeartFailure

In [24]:
trIclonB= trIclon.merge(trB, on='BeneID')
trOclonB= trOclon.merge(trB, on='BeneID')

In [26]:
conditions_list=['RenalDiseaseIndicator','ChronicCond_Alzheimer',
       'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
       'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
       'ChronicCond_Depression', 'ChronicCond_Diabetes',
       'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
       'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke']

conditionsShort_list=['ESRD','Alzheimer',
       'HeartFailure', 'KidneyDisease',
       'Cancer', 'COPD',
       'Depression', 'Diabetes',
       'IschemicHeart', 'Osteoporosis',
       'RheumatoidArthritis', 'Stroke']

conditionsMin_list=['ESRD','Alzh',
       'HF', 'KD',
       'Canc', 'COPD',
       'Depr', 'Diab',
       'IH', 'Oste',
       'RA', 'Stro']

In [42]:
# one hot encode by replacing not chronic condition encoding '2' with 0 
trIclonBc = trIclonB[conditions_list].replace([2], 0)

In [30]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [45]:
# The support is computed as the fraction: transactions_where_item(s)_occur / total_transactions.
frequent_itemsets= apriori(trIclonBc, min_support=0.4, use_colnames=True) # itemsets appearing in at least x% of claims
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.433311,(ChronicCond_Alzheimer),1
1,0.642336,(ChronicCond_Heartfailure),1
2,0.525547,(ChronicCond_KidneyDisease),1
3,0.410750,(ChronicCond_ObstrPulmonary),1
4,0.442601,(ChronicCond_Depression),1
5,0.753152,(ChronicCond_Diabetes),1
6,0.820173,(ChronicCond_IschemicHeart),1
7,0.400796,"(ChronicCond_KidneyDisease, ChronicCond_Heartf...",2
8,0.520902,"(ChronicCond_Diabetes, ChronicCond_Heartfailure)",2
9,0.568016,"(ChronicCond_IschemicHeart, ChronicCond_Heartf...",2


# Distribution of how many times cloning occurs (number of cloned rows)

In [83]:
trIclon['AttendingPhysician'].value_counts()

PHY422134    21
PHY341560    14
PHY412508    10
PHY362864     8
PHY329774     8
             ..
PHY382067     1
PHY399415     1
PHY397567     1
PHY374106     1
PHY375117     1
Name: AttendingPhysician, Length: 1091, dtype: int64

In [82]:
trIclon['Provider'].value_counts()

PRV52019    22
PRV55462    21
PRV54367    16
PRV55209    14
PRV52340    14
            ..
PRV55208     1
PRV51579     1
PRV55755     1
PRV51533     1
PRV57433     1
Name: Provider, Length: 716, dtype: int64

In [91]:
trIclon.sort_values(by=['ClmDiagnosisCode_1']).tail(200)

,index,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
21531,21531,BENE89771,CLM71359,2009-10-09,2009-10-09,PRV55209,4000,PHY341560,0,0,2009-10-09,78659,1068.0,2009-10-09,239,78659,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4521,4521,BENE27158,CLM54715,2009-06-08,2009-06-13,PRV52340,2000,PHY412160,PHY312729,0,2009-06-08,78650,1068.0,2009-06-13,257,78659,53550,515,41401,29640,41402,4019,4414,2767,0,3995.0,0,0,0,0,0
7065,7065,BENE36637,CLM76907,2009-11-22,2009-11-25,PRV55049,2000,PHY391949,PHY385176,0,2009-11-22,78650,1068.0,2009-11-25,221,78659,3089,5119,2449,5533,2859,72190,4148,40391,0,9390.0,0,0,0,0,0
37056,37056,BENE146791,CLM54404,2009-06-06,2009-06-11,PRV57049,2000,PHY400672,PHY431011,0,2009-06-06,78650,1068.0,2009-06-11,246,78659,53081,4940,2948,V4582,2859,486,28260,4148,0,9205.0,0,0,0,0,0
27340,27340,BENE111218,CLM37400,2009-02-07,2009-02-09,PRV57129,2000,PHY353685,0,0,2009-02-07,78650,1068.0,2009-02-09,218,78659,4019,41401,42781,0,0,0,0,0,0,0,0,0,0,0,0
22615,22615,BENE93782,CLM70404,2009-10-02,2009-10-05,PRV53602,7000,PHY372373,PHY386549,0,2009-10-02,78650,1068.0,2009-10-05,247,78659,4019,3051,41401,2720,4111,43310,71590,V5861,4254,3722.0,0,0,0,0,0
32944,32944,BENE131524,CLM72128,2009-10-15,2009-10-16,PRV57223,2000,PHY349912,0,0,2009-10-15,78659,1068.0,2009-10-16,248,78659,0414,412,340,V4582,2724,V173,V1582,25000,0,0,0,0,0,0,0
17354,17354,BENE74764,CLM41070,2009-03-04,2009-03-07,PRV55462,7000,PHY422134,PHY429430,0,2009-03-04,78650,1068.0,2009-03-07,241,78659,4019,3051,41401,2720,4111,43310,71590,V5861,4254,3722.0,0,0,0,0,0
25210,25210,BENE103474,CLM53426,2009-05-30,2009-06-02,PRV54894,7000,PHY340335,0,0,2009-05-30,78701,1068.0,2009-06-02,357,78791,25000,V1582,40391,V1251,7824,71590,99681,5990,0,0,0,0,0,0,0
24441,24441,BENE100760,CLM47100,2009-04-15,2009-04-18,PRV54565,7000,PHY314924,0,0,2009-04-15,78701,1068.0,2009-04-18,388,78791,25000,V1582,40391,V1251,7824,71590,99681,5990,0,0,0,0,0,0,0


In [87]:
trIclon[trIclon.duplicated(subset=diag+proc, keep='first')]

,index,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
35,35,BENE11093,CLM82202,2009-01-03,2009-01-12,PRV51842,20000,PHY341192,PHY355416,0,2009-01-03,4439,1068.0,2009-01-09,255,99662,2639,E8782,920,V854,4280,E9320,04185,412,42732,3950.0,2762.0,0,0,0,0
766,766,BENE13844,CLM41350,2009-03-06,2009-03-08,PRV56260,4000,PHY347493,PHY342803,0,2009-03-06,60000,1068.0,2009-03-08,714,60000,25000,5960,185,78820,5968,496,2720,V1582,0,6029.0,0,0,0,0,0
1229,1229,BENE15306,CLM77516,2009-11-27,2009-12-05,PRV56469,25000,PHY319781,PHY338861,0,2009-11-27,436,1068.0,2009-12-05,064,33520,4280,4019,2869,44024,25000,25070,V4501,2851,0,8411.0,0,0,0,0,0
1567,1567,BENE16452,CLM36876,2009-02-04,2009-02-06,PRV51516,3000,PHY344681,0,0,2009-02-04,7802,1068.0,2009-02-06,054,4359,53782,4019,496,2724,5990,42731,99811,V160,0,0,0,0,0,0,0
1577,1577,BENE16511,CLM66050,2009-08-31,2009-09-09,PRV53401,6000,PHY370379,PHY407678,PHY327384,2009-08-31,42843,1068.0,2009-09-09,216,42823,4240,4280,25000,4148,2662,V5861,41400,27800,0,8872.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40361,40361,BENE158810,CLM39286,2009-02-20,2009-02-21,PRV53762,2000,PHY378430,0,0,2009-02-20,29181,1068.0,2009-02-21,894,29181,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40388,40388,BENE158883,CLM35368,2009-01-24,2009-01-27,PRV55435,10000,PHY403345,PHY403345,0,2009-01-24,79902,1068.0,2009-01-27,868,0389,99592,48241,78552,5859,2809,5856,2768,27651,0,9905.0,0,0,0,0,0
40411,40411,BENE158947,CLM43434,2009-03-20,2009-03-25,PRV57204,0,PHY367487,PHY329719,0,2009-03-20,486,1068.0,2009-03-25,182,4809,5990,25000,42823,42822,2724,34400,7392,4280,0,9952.0,0,0,0,0,0
40413,40413,BENE158952,CLM36936,2009-02-04,2009-02-07,PRV54342,7000,PHY417489,PHY353365,0,2009-02-04,78650,1068.0,2009-02-07,443,57410,36250,53081,5739,42789,4280,4293,4019,5531,0,5123.0,0,0,0,0,0
